In [2]:
!pip install -U abacusai

     |████████████████████████████████| 3.4 MB 5.1 MB/s 
     |████████████████████████████████| 2.3 MB 38.4 MB/s 
  Created wheel for abacusai: filename=abacusai-0.36.4-py3-none-any.whl size=134875 sha256=790546d892e563664aceb95669e43e747669918ea0194c6a8a452998c470f130
  Stored in directory: /root/.cache/pip/wheels/f5/30/4e/1f7d42e339242d63030a7622906e7377dbaff762bcafa211be
Successfully built abacusai


In [3]:
import collections
import threading
import time

from tqdm import tqdm
import numpy as np
import IPython.display

import abacusai

# Set up API client and load data

In [4]:
api_key = '9aaf73ff552c4914b6a4642f46115cdc'
server = 'https://abacus.ai'

In [5]:
api_client = abacusai.ApiClient(api_key=api_key, server=server)
api_client

In [6]:
feature_group = api_client.describe_feature_group('10c6c17150')
data = feature_group.load_as_pandas()
data.head()

,vehicle_title,review_title,review
0,1997 Toyota Previa Minivan LE All-Trac 3dr Min...,"my 4th previa, best van ever made!",1st 95 went over 300k before being totalled b...
1,1997 Toyota Previa Minivan LE 3dr Minivan,Mom's Taxi Babies Ride,Sold 86 Toyota Van 285K miles to be replaced ...
2,1997 Toyota Previa Minivan LE All-Trac 3dr Min...,Best Minivan ever,My 1997 AWD Previa is the third one that I ha...
3,1997 Toyota Previa Minivan LE All-Trac 3dr Min...,Final model year of the great Previa,"An amazing vehicle: mid-engine, supercharged,..."
4,2007 Toyota Avalon Sedan XLS 4dr Sedan (3.5L 6...,I'll drive this car until they take it away fr...,Bought this Avy in 2007 used with 1200 miles ...


# Get predictions from sentiment model

In [8]:
from abacusai import PredictionClient
client = PredictionClient()
client.get_sentiment(deployment_token='dd6f423c5956422abf2c6cd07c21b071', deployment_id='643d34f0c', document=" Toyotas will run forever and hybrid design is solid.The car is quiet and the seats are great.The interior build quality is terrible however.My headliner warped in the first 4K miles, the dash creaks and pops when you hit a dump and the door panels also creak constantly.A happy customer tells four people about their experience and unhappy customer tells 10 people.The crappy part is I have to sell it at a loss to buy the ford fusion.My gut told me to go with the ford and I should have followed it.")

{'admiration': 0.18734998114836707,
 'amusement': 0.00015829608571614316,
 'anger': 0.001131381759855807,
 'annoyance': 0.09705089982431851,
 'approval': 0.2514723889317682,
 'caring': 0.0007489281076559976,
 'confusion': 0.0007848413354010962,
 'curiosity': 0.00039328829884743366,
 'desire': 0.015303365088136841,
 'disappointment': 0.8165182041561381,
 'disapproval': 0.00733822621012426,
 'disgust': 0.0005704679312483601,
 'embarrassment': 0.00011633836868711977,
 'excitement': 0.0012685498495989575,
 'fear': 0.0002763517529304868,
 'gratitude': 0.00023449238225218384,
 'grief': 0.0007154216040626338,
 'joy': 0.0019889361907007015,
 'love': 0.00025728372515842725,
 'nervousness': 0.0010029814916877676,
 'neutral': 0.10354925047401577,
 'optimism': 0.008917951741144722,
 'pride': 0.0062754131669841,
 'realization': 0.019385494946991308,
 'relief': 0.0024816415701369478,
 'remorse': 0.00281108329032291,
 'sadness': 0.007259515837988566,
 'surprise': 0.00025386384160559593}

In [ ]:
sent_deployment_token = '5a9b51df6b6f497a9d75487fb8b42131'
sent_deployment_id = '3714f6bd0'

In [ ]:
tqdm._instances.clear()
sample_texts = data['review'][:100]
sentiments = [
    client.get_sentiment(
        deployment_token=sent_deployment_token,
        deployment_id=sent_deployment_id,
        document=text
    )
    for text in tqdm(sample_texts)
]

100%|██████████| 100/100 [00:41<00:00,  2.38it/s]


In [ ]:
query = 'joy'
print(f'Top scoring texts for: "{query}"\n')
scores = [s[query] for s in sentiments]
arg_sort = np.argsort(-np.array(scores))
for i in arg_sort[:5]:
    print(scores[i])
    print(sample_texts[i])
    print('')

Top scoring texts for: "joy"



KeyError: ignored

In [ ]:
query = 'annoyance'
print(f'Top scoring texts for: "{query}"\n')
scores = [s[query] for s in sentiments]
arg_sort = np.argsort(-np.array(scores))
for i in arg_sort[:5]:
    print(scores[i])
    print(sample_texts[i])
    print('')

In [ ]:
query = 'fear'
print(f'Top scoring texts for: "{query}"\n')
scores = [s[query] for s in sentiments]
arg_sort = np.argsort(-np.array(scores))
for i in arg_sort[:5]:
    print(scores[i])
    print(sample_texts[i])
    print('')

# Create a classification model

In [ ]:
print('Creating project')
class_project = api_client.create_project('nlp_classification', use_case='NLP_CLASSIFICATION')

api_client.add_feature_group_to_project(
    feature_group_id=feature_group, project_id=class_project, feature_group_type='DOCUMENTS'
)

class_project.set_feature_mapping(
    feature_group_id=feature_group,
    feature_name='review',
    feature_mapping='DOCUMENT',
)

print('Training model')
class_model = class_project.train_model(
    name='classification_model_1',
    training_config={
        'zero_shot_hypotheses': [
            'This text is about car speed / acceleration / slowness',
            'This text is about car price / cost / value for money',
            'This text is about car build quality',
            'This text is about car seats',
        ]
    }
)

class_model = class_model.wait_for_full_automl()
print('Deploying model')
class_deployment = api_client.create_deployment(model_id=class_model)
class_deployment = class_deployment.wait_for_deployment()
class_deployment_token = class_project.create_deployment_token()
class_deployment_id = class_deployment.deployment_id

print('Model deployed')

Creating project
Training model
Deploying model
Model deployed


In [1]:
print('Testing prediction')
client.get_classification(
    deployment_token=class_deployment_token,
    deployment_id=class_deployment_id,
    document="This car is a bargain."
)

sample_texts = data['review'][:100]
text_classifications = []

text_to_prediction = {}

print('Producing predictions')

# Produce predictions in the background so we can do some analysis straight away

def process_data(texts):
    for text in texts:
        text_to_prediction[text] = client.get_classification(
            deployment_token=class_deployment_token,
            deployment_id=class_deployment_id,
            document=text,
        )
        
thread = threading.Thread(target=process_data, args=(sample_texts,))
thread.start()

while len(text_to_prediction) < 20:
    IPython.display.clear_output(wait=True)
    print(f'Predictions produced so far: {len(text_to_prediction)}')
    time.sleep(1)
print('Example prediction:')
list(text_to_prediction.values())[0]

Testing prediction


NameError: ignored

In [ ]:
predictions = []
for text in sample_texts:
    if text not in text_to_prediction:
        break
    else:
        predictions.append(text_to_prediction[text])
print(f'Predictions made so far: {len(predictions)}\n')
query = list(predictions[0].keys())[1]
print(f'Top scoring texts for: "{query}"\n')
scores = [s[query] for s in predictions]
arg_sort = np.argsort(-np.array(scores))
for i in arg_sort[:5]:
    print(f'Score = {scores[i]}')
    print(sample_texts[i])
    print('')

# Build a named entity recognition (NER) model

In [ ]:
ner_project = api_client.create_project('named_entity_recognition', use_case='NAMED_ENTITY_RECOGNITION')

api_client.add_feature_group_to_project(
    feature_group_id=feature_group, project_id=ner_project, feature_group_type='DOCUMENTS'
)

ner_project.set_feature_mapping(
    feature_group_id=feature_group,
    feature_name='review',
    feature_mapping='DOCUMENT',
)

ner_model = ner_project.train_model(
    name='ner_model_1',
    training_config={'NER_MODEL_TYPE': "pretrained uncased 43 categories"}
)

ner_model.wait_for_full_automl()

ner_deployment = api_client.create_deployment(model_id=ner_model)

ner_deployment = ner_deployment.wait_for_deployment()

ner_deployment_token = ner_project.create_deployment_token()

ner_deployment_id = ner_deployment.deployment_id
ner_deployment_token = ner_deployment_token

client.get_labels(deployment_token=ner_deployment_token, deployment_id=ner_deployment_id, query_data={"content":"I love my Toyota Camry!"})

In [ ]:
tqdm._instances.clear()
sample_texts = data['review'][:100]
annotations = [
    client.get_labels(deployment_token=ner_deployment_token, deployment_id=ner_deployment_id, query_data={'content': text})['annotations']
    for text in tqdm(sample_texts)
]

In [ ]:
label_counts = collections.Counter([anno['displayName'] for anno_list in annotations for anno in anno_list])
label_counts.most_common(10)

In [ ]:
product_counts = collections.Counter([anno['textExtraction']['textSegment']['phrase'].strip().lower()
                                      for anno_list in annotations for anno in anno_list if anno['displayName'] == 'product'])
product_counts.most_common(10)

# Combined analysis using multiple models

In [ ]:
# Filter using NER

filtered_texts = [
    text
    for text, anno_list in zip(sample_texts, annotations)
    if any([anno['displayName'] == 'product' and 'avalon' in anno['textExtraction']['textSegment']['phrase'].strip().lower()
           for anno in anno_list])
]
len(filtered_texts)

In [ ]:
# See top scores after the filter

predictions = []
for text in filtered_texts:
    if text not in text_to_prediction:
        break
    else:
        predictions.append(text_to_prediction[text])
query = list(predictions[0].keys())[1]
print(f'Top scoring texts for: "{query}"\n')
scores = [s[query] for s in predictions]
arg_sort = np.argsort(-np.array(scores))
for i in arg_sort[:5]:
    print(scores[i])
    print(filtered_texts[i])
    print('')